In [2]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip wwm_uncased_L-24_H-1024_A-16.zip

!ls wwm_uncased_L-24_H-1024_A-16

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

!ls uncased_L-12_H-768_A-12

!pip install simpletransformers
!pip install transformers
!pip install tensorboardx
!pip install ktrain

--2020-06-29 00:31:47--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.217.48, 172.217.3.208, 172.217.14.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.217.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  71.1MB/s    in 18s     

2020-06-29 00:32:04 (67.7 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip’ saved [1248381879/1248381879]

Archive:  wwm_uncased_L-24_H-1024_A-16.zip
   creating: wwm_uncased_L-24_H-1024_A-16/
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating:

In [3]:
!pip install bert
!pip install bert-tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2

     |████████████████████████████████| 40 kB 3.4 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=ee65aaf4a9d7268172287e3ac291ebff53df687e0b1e28716a6f20847db5e469
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/b5/ea/3e768502fa1032a6bfefcd0e7dd95e2110e026bf3c584940cd
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=b263621e384c0a996b453edd18a1bf7ea2d125490ba3c00400e2da9c06ee6110
  Stored in directory: /home/ec2-user/.cache/pip/wheels/18/e2/80/244168a554fc8469897bb86b3ea7183b56ecc92f484e67b375
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=52e2d2363fa9418fdeb4c70715c9a7ad1c311686249c8a571b0a0b218b3b800c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9d/c6/c5/a3f4a618640f461130f47e76703156c13a39cc3b96b5f4f439
Successfully built bert-for-tf2 py-params params-flow
You should consider upgrading via the '/home/ec2-use

In [4]:
import pandas as pd
import re
import string
import bert
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import train_test_split
import os
#from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras import layers
import tqdm
from tensorflow.keras import layers
import pickle
import math
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [2]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [3]:
def tokenize_claims(text_claims):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_claims))

In [8]:
def create_model(max_seq_len,cnn_filters,dropout_rate,dnn_units):
    input_ids = keras.layers.Input(
                                   shape=(max_seq_len, ),
                                   dtype='int32',
                                   name="input_ids"
                                   )
    input_mask = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_mask")
    segment_ids = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="segment_ids")
    
    _, bert_output = bert_layer([input_ids, input_mask, segment_ids])
    #bert_output = bert_layer(input_ids)
    print("bert shape", bert_output.shape)
    
    
    cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer1 = layers.GlobalMaxPool1D()(cnn_layer1)
    cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer2 = layers.GlobalMaxPool1D()(cnn_layer2)
    cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer3 = layers.GlobalMaxPool1D()(cnn_layer3)

    concatenated = tf.concat([cnn_layer1, cnn_layer2, cnn_layer3], axis=-1) # (batch_size, 3 * cnn_filters)   
    dense_1 = layers.Dense(units=dnn_units, activation="relu")(concatenated)
    dropout = layers.Dropout(rate=dropout_rate)(concatenated)
       
    last_dense = layers.Dense(units=1,activation="sigmoid")(dropout)
    
    model = keras.Model(inputs=[input_ids,input_mask,segment_ids], outputs=last_dense)
    
    #cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    #cls_out = keras.layers.Dropout(0.5)(cls_out)
    #logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    #logits = keras.layers.Dropout(0.5)(logits)
    #logits = keras.layers.Dense(
    #units=len(classes),
    #activation="softmax"
    #)(logits)
    #model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))
    #load_stock_weights(bert, bert_ckpt_file)
    return model

In [19]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
bucket='trainedbertmodelweights'
data_key = 'BERT_EMBEDDINGS_TRAINABLE_CNN_weights-improvement-19-0.98.hdf5'
weights_location = 's3://{}/{}'.format(bucket, data_key)
print(weights_location)

s3_client = boto3.client('s3')
s3_client.download_file(bucket, data_key, 'hello2.txt')

s3://trainedbertmodelweights/BERT_EMBEDDINGS_TRAINABLE_CNN_weights-improvement-19-0.98.hdf5


In [20]:
precision_obj = tf.keras.metrics.Precision()
recall_obj = tf.keras.metrics.Recall()
#Get model for Experiment3 created
CNN_FILTERS = 200
DROPOUT_RATE = 0.1
DNN_UNITS = 128
model = create_model(60,CNN_FILTERS,DROPOUT_RATE,DNN_UNITS)
#load model weights
model.load_weights("./weights.hdf5")

model.compile(
               optimizer=keras.optimizers.Adam(1e-5),
               loss='binary_crossentropy',
               metrics=['accuracy',precision_obj,recall_obj]
             )

bert shape (None, None, 768)


In [21]:
def predict_claims(claim_text):
    claims = [preprocess_text(claim_text)]

    max_seq_len=60
    x_input,masks,segments = [], [],[]
    for new_claim in claims:
        text = tokenizer.tokenize(new_claim)
        text = text[:max_seq_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_seq_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_seq_len
        x_input.append(np.array(tokens))
        masks.append(np.array(pad_masks))
        segments.append(np.array(segment_ids))

    return model.predict(x = [x_input,masks,segments])[0]

In [22]:
predict_claims('masks spread the coronavirus')[0]

0.00062717195